In [1]:
import argparse
from LoadData_nlp_new import *
from DataSet_nlp_new import *
from torch.utils.data import DataLoader
from TwitterModel_nlp import *

C:\Users\19915\ai\anaconda3\envs\pytorch\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
from transformers import BertModel

bert_model = BertModel.from_pretrained('bert-base-uncased')

print("Done loading BERT model.")

In [3]:
train_tweets_file_path = "../data/all_data/train_data/"
tweet_data_train_txt = "../data/project-data/train.data.txt"
train_label = "../data/project-data/train.label.txt"
dev_tweets_file_path = "../data/all_data/dev_data/"
tweet_data_dev_txt = "../data/project-data/dev.data.txt"
dev_label =  "../data/project-data/dev.label.txt"
load_twitter_train_data = LoadDataAndProcessing(train_tweets_file_path,tweet_data_train_txt,train_label)
#load_twitter_train_data = LoadDataAndProcessing(train_tweets_file_path,tweet_data_train_txt)
load_twitter_dev_data = LoadDataAndProcessing(dev_tweets_file_path,tweet_data_dev_txt,dev_label)

train_input = load_twitter_train_data.loadData()
dev_input = load_twitter_dev_data.loadData()

FileNotFoundError: [Errno 2] No such file or directory: '../data/all_data/train_data/554894001946759168.json'

In [ ]:
train_input

In [ ]:
trainset = TaskOneDataset(train_input)
devset = TaskOneDataset(dev_input)

In [ ]:
trainloader = DataLoader(trainset, batch_size=1, collate_fn=trainset.create_mini_batch)
devloader = DataLoader(devset, batch_size=1, collate_fn=devset.create_mini_batch)

In [ ]:
from transformers import BertTokenizer, BertForPreTraining
import torch

In [ ]:
from transformers import BertForSequenceClassification

PRETRAINED_MODEL_NAME = "bert-base-uncased"
NUM_LABELS = 2

model = BertForSequenceClassification.from_pretrained(
    PRETRAINED_MODEL_NAME, num_labels=2)

In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    
    model.eval()  # 推論模式
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            outputs = model(*data[:3])
            # 前 3 個 tensors 分別為 tokens, segments 以及 masks
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
_, acc = get_predictions(model, trainloader, compute_acc=True)
print("classification acc:", acc)

In [ ]:
def get_learnable_params(module):
    return [p for p in module.parameters() if p.requires_grad]
    
model_params = get_learnable_params(model)
optimizer = torch.optim.Adam(model_params, lr=1.0e-5)

In [ ]:
import time
start = time.time()

import matplotlib.pyplot as plt
train_acc = []

EPOCHS = 20
for epoch in range(EPOCHS):
    
    running_loss = 0.0
    for data in trainloader:
        
        tokens_tensors, segments_tensors, \
        masks_tensors, labels = [t.to(device) for t in data]

        optimizer.zero_grad()
        # forward pass
        outputs = model(input_ids=tokens_tensors, 
                        token_type_ids=segments_tensors, 
                        attention_mask=masks_tensors, 
                        labels=labels)
        loss = outputs[0]
        # backward
        loss.backward()
        optimizer.step()
        
        # 紀錄當前 batch loss
        running_loss += loss.item()
        
    # 計算分類準確率
    _, acc = get_predictions(model, trainloader, compute_acc=True)
    train_acc.append(acc)

    print(f"batch size:1")
    print(f'[epoch {epoch+1}] loss: {running_loss:3f}, acc: {acc:3f}')

end = time.time()
print(f"time:{end-start:.2f}")

In [ ]:
class LoadTestAndProcessing():
    def __init__(self,tweets_file_path,tweet_data):
        self.tweets_file_path = tweets_file_path
        self.tweet_data = tweet_data

    def textProcess(self,text):
        text = re.sub("@[\w]*", "", text)  # 去@
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', "", text)  # 去URL
        text = re.sub('[\n\t-]', '', text)
        text = text.strip()
        return text
    def get_tweet_list(self,tweets_file_path,id_list):
        text_list = []
        for i in id_list:
            file_name = tweets_file_path + i + ".json"
            with open(file_name) as f:
                json_data = json.load(f)
            text_list.append(json_data["text"])
        return text_list
    
    def loadData(self):
        all_ids = pd.read_table(tweet_data,sep='\n',header=None)
        data = []
        for i in range(len(all_ids)):
            list_id = list(all_ids.iloc[i])
            list_sub_id = list_id[0].split(",")
            text_list_sub =  get_tweet_list(list_sub_id)
            data.append(text_list_sub)
        tweeter_source = []
        tweeter_replay = []
        tweeter_id = []
        text_p1 = ""
        text_p2 = ""
        no_source = 0
        for tweet_data_list in data:
            temp_source = ""
            temp_reply = []
            source_tweet = tweet_data_list[0]['text']
            source_num = 0
            for tweet in tweet_data_list:
                if  'in_reply_to_user_id' not  in tweet.keys():
                    source_num = source_num + 1
            if  source_num == 0:
                text_p1 = self.textProcess(source_tweet['text']).lower()
                temp_source = temp_source + source_tweet
                tweeter_id.append(source_tweet['id_str'])
            elif  source_num > 1:
                for tweet in tweet_data_list:
                    if  'in_reply_to_user_id' not  in tweet.keys():
                        text_p1 = self.textProcess(tweet['text']).lower()
                        temp_source = temp_source + text_p1
                        tweeter_id.append(tweeter['id_str'])
                    else:
                        text_p2 = self.textProcess(tweet['text']).lower()
                        res_sim = difflib.SequenceMatcher(None, text_p1, text_p2).quick_ratio()
                        if res_sim >= 0.93:
                            pass
                        else:
                            temp_reply.append(text_p2)
            else:
                for tweet in tweet_data_list:
                    if  'in_reply_to_user_id' not  in tweet.keys():
                        text_p1 = self.textProcess(tweet['text']).lower()
                        temp_source = temp_source + text_p1
                else:
                    text_p2 = self.textProcess(tweet['text']).lower()
                    res_sim = difflib.SequenceMatcher(None, text_p1, text_p2).quick_ratio()
                    if res_sim >= 0.93:
                        pass
                    elif res_sim <= 0.01:
                        pass
                    else:
                        temp_reply.append(text_p2)
            no_source = no_source +1    
            tweeter_source.append(temp_source)                          
            tweeter_replay.append(temp_reply)     

        total_replay = []
        for i in tweeter_replay:
            temp = ''
            for line in i:
                temp += line
            total_replay.append(temp)

        input_value = []
        for i in range(len(label)):
            temp = {}
            temp['text'] = tweeter_source[i]
            temp['textb'] = total_replay[i]
            input_value.append(temp)

        return input_value,tweeter_id

In [ ]:
class LoadTestAndProcessing():
    def __init__(self,tweets_file_path,tweet_data):
        self.tweets_file_path = tweets_file_path
        self.tweet_data = tweet_data

    def textProcess(self,text):
        text = re.sub("@[\w]*", "", text)  # 去@
        text = re.sub(r'(https|http)?:\/\/(\w|\.|\/|\?|\=|\&|\%)*\b', "", text)  # 去URL
        text = re.sub('[\n\t-]', '', text)
        text = text.strip()
        return text
    def get_tweet_list(self,tweets_file_path,id_list):
        text_list = []
        for i in id_list:
            file_name = tweets_file_path + i + ".json"
            with open(file_name) as f:
                json_data = json.load(f)
            text_list.append(json_data)
        return text_list
    
    def loadData(self):
        all_ids = pd.read_table(tweet_data,header=None)
        data = []
        for i in range(len(all_ids)):
            list_id = list(all_ids.iloc[i])
            list_sub_id = list_id[0].split(",")
            text_list_sub =  self.get_tweet_list(tweets_file_path,list_sub_id)
            data.append(text_list_sub)
        #print(data[0])
        tweeter_source = []
        tweeter_replay = []
        tweeter_id = []
        text_p1 = ""
        text_p2 = ""
        no_source = 0
        for tweet_data_list in data:
            temp_source = ""
            temp_reply = []
            #print(tweet_data_list[0])
            source_tweet = tweet_data_list[0]
            source_num = 0
            for tweet in tweet_data_list:
                if  not tweet['in_reply_to_status_id']:
                    source_num = source_num + 1
            if  source_num == 0:
                text_p1 = self.textProcess(source_tweet['text']).lower()
                temp_source = temp_source + text_p1
                tweeter_id.append(source_tweet['id_str'])
            elif  source_num > 1:
                for tweet in tweet_data_list:
                    if  not tweet['in_reply_to_status_id']:
                        text_p1 = self.textProcess(tweet['text']).lower()
                        temp_source = temp_source + text_p1
                        tweeter_id.append(tweet['id_str'])
                    else:
                        text_p2 = self.textProcess(tweet['text']).lower()
                        res_sim = difflib.SequenceMatcher(None, text_p1, text_p2).quick_ratio()
                        if res_sim >= 0.93:
                            pass
                        else:
                            temp_reply.append(text_p2)
            else:
                for tweet in tweet_data_list:
                    if  not tweet['in_reply_to_status_id']:
                        text_p1 = self.textProcess(tweet['text']).lower()
                        temp_source = temp_source + text_p1
                    else:
                        text_p2 = self.textProcess(tweet['text']).lower()
                        res_sim = difflib.SequenceMatcher(None, text_p1, text_p2).quick_ratio()
                        if res_sim >= 0.93:
                            pass
                        elif res_sim <= 0.01:
                            pass
                        else:
                            temp_reply.append(text_p2)
            no_source = no_source +1    
            tweeter_source.append(temp_source)                          
            tweeter_replay.append(temp_reply)     

        total_replay = []
        for i in tweeter_replay:
            temp = ''
            for line in i:
                temp += line
            total_replay.append(temp)

        input_value = []
        for i in range(len(tweeter_source)):
            temp = {}
            temp['text'] = tweeter_source[i]
            temp['textb'] = total_replay[i]
            input_value.append(temp)

        return input_value,tweeter_id

In [ ]:
import pandas as pd
import json
import re
import difflib

tweets_file_path = "../data/project-data/tweet-objects/tweet-objects/"
tweet_data = "../data/project-data/test.data.txt"
load_twitter_test_data= LoadTestAndProcessing(tweets_file_path,tweet_data)

test_input,twitter_id  = load_twitter_test_data.loadData()

In [ ]:
testset = TaskTestDataset(test_input)
testloader = DataLoader(testset, batch_size=1, collate_fn=testset.create_mini_batch)

In [ ]:
def predict(model,dataloader):
    result_list = []
    with torch.no_grad():
        for data in dataloader:
            tokens_tensors, segments_tensors, masks_tensors, labels = [t.to(device) for t in data]
            optimizer.zero_grad()
            # forward pass
            outputs = model(input_ids=tokens_tensors, 
                            token_type_ids=segments_tensors, 
                            attention_mask=masks_tensors, 
                            labels=labels)

    return result_list

In [ ]:
def get_predictions(model, dataloader, compute_acc=False):
    predictions = None
    correct = 0
    total = 0
    
    model.eval()  # 推論模式
    with torch.no_grad():
        # 遍巡整個資料集
        for data in dataloader:
            # 將所有 tensors 移到 GPU 上
            if next(model.parameters()).is_cuda:
                data = [t.to("cuda:0") for t in data if t is not None]
            
            outputs = model(*data[:3])
            # 前 3 個 tensors 分別為 tokens, segments 以及 masks
            logits = outputs[0]
            _, pred = torch.max(logits.data, 1)
            
            # 用來計算訓練集的分類準確率
            if compute_acc:
                labels = data[3]
                total += labels.size(0)
                correct += (pred == labels).sum().item()
                
            # 將當前 batch 記錄下來
            if predictions is None:
                predictions = pred
            else:
                predictions = torch.cat((predictions, pred))
    
    if compute_acc:
        acc = correct / total
        return predictions, acc
    return predictions

# 讓模型跑在 GPU 上並取得訓練集的分類準確率
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print("device:", device)
model = model.to(device)
pred = get_predictions(model, testloader, compute_acc=False)
#print("classification acc:", acc)

In [ ]:
pred_list = pred.tolist()

In [ ]:
new_list = []
for i in pred_list:
    if i == 0:
        new_list.append(1)
    else:
        new_list.append(0)

In [ ]:
import pandas as pd
test=pd.DataFrame(data=new_list)
print(test)
import csv
n = len(test)
nlist = range(0,n)
test['Id'] = nlist
test.columns = ['Predicted','Id']
test[['Id','Predicted']] = test[['Predicted','Id']]
test.columns = ['Id','Predicted']
test.to_csv('../data/test_nlp_v2.csv',encoding='gbk',index = False, quoting=csv.QUOTE_NONNUMERIC)

In [ ]:
len(pred) 